In [ ]:
import tensorflow as tf
import zipfile, os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

# Download dataset
zip_path = tf.keras.utils.get_file(
    'cats_and_dogs_filtered.zip',
    origin='https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
)

# Extract manually
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

# Correct paths
data_dir = '/content/cats_and_dogs_filtered'
train_dir = os.path.join(data_dir, 'train')
val_dir   = os.path.join(data_dir, 'validation')

print("Train Dir Exists:", os.path.exists(train_dir))
print("Val Dir Exists:", os.path.exists(val_dir))

# Data generators
datagen = ImageDataGenerator(rescale=1./255)
train_gen = datagen.flow_from_directory(train_dir, target_size=(160,160), batch_size=16, class_mode='binary')
val_gen   = datagen.flow_from_directory(val_dir,   target_size=(160,160), batch_size=16, class_mode='binary')

# Transfer Learning Model
base = ResNet50(input_shape=(160,160,3), include_top=False, weights='imagenet')
base.trainable = False

model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_gen, epochs=3, validation_data=val_gen)

print("✅ Done")




Train Dir Exists: True
Val Dir Exists: True
Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 62s 458ms/step - accuracy: 0.8688 - loss: 0.2855 - val_accuracy: 0.9510 - val_loss: 0.1163
Epoch 2/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 57s 455ms/step - accuracy: 0.9742 - loss: 0.0780 - val_accuracy: 0.9580 - val_loss: 0.1058
Epoch 3/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 56s 447ms/step - accuracy: 0.9801 - loss: 0.0609 - val_accuracy: 0.9620 - val_loss: 0.0951
✅ Done


In [ ]:
# ✅ Unfreeze some layers to fine-tune and compare performance
base.trainable = True
for layer in base.layers[:-30]:   # unfreeze last 30 layers only
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy', metrics=['accuracy'])

history_ft = model.fit(train_gen, epochs=2, validation_data=val_gen)

print("✅ Fine-tuning done")


Epoch 1/2
125/125 ━━━━━━━━━━━━━━━━━━━━ 84s 600ms/step - accuracy: 0.8679 - loss: 0.3302 - val_accuracy: 0.9590 - val_loss: 0.1092
Epoch 2/2
125/125 ━━━━━━━━━━━━━━━━━━━━ 74s 597ms/step - accuracy: 0.9472 - loss: 0.1390 - val_accuracy: 0.9600 - val_loss: 0.1090
✅ Fine-tuning done
